In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from pprint import pprint
import json 

## Start scraping process

### Getting all navigation of the website for access the elements

In [2]:
url = 'https://www.realestate.com.kh/buy/?page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}

category_url = []

response = requests.get(url, headers=headers)
data = bs(response.content, 'html.parser')

# query all headers
get_all_header = data.find_all(class_='items')
get_all_header[0]

print(get_all_header[0])
# for i in get_all_header[0]:
#     print(i.find(class_='value').get_text())



<div class="items"><div class="item css-1uuzwjq eq4or9x0"><header class="featured css-ineky e1jqslr40"><a href="/buy/kandaek/properties-dabest-153486/" target="_blank">Villa for Sale in Siem Reap-Kon Deak Borey Tourism siem reap</a></header><article class="shadow featured css-1dego4x e1gd26e40"><div class="labels"><div class="featured" style="background-color:#8DCA3F">Featured</div><div class="translucent">For sale</div></div><a class="carousel-wrap" href="/buy/kandaek/properties-dabest-153486/" target="_blank"><div class="css-m6v9vd"><div class="css-g96nnt"><div class="carousel-scrollbox css-ifzriq"><div class="carousel-item"><div class="carousel-img-wrap"><script style="display:none" type="text/javascript">
//<![CDATA[
window.__mirage2 = {petok:"f57Rb.eSfasrGRp2LYcuVFK9qr1cxnjQahPPnUfkTE0-1800-0"};
//]]>
</script>
<script src="https://ajax.cloudflare.com/cdn-cgi/scripts/04b3eb47/cloudflare-static/mirage2.min.js" type="text/javascript"></script>
<img alt="1Y3A7065.jpg" data-cfsrc="htt

In [3]:
print(get_all_header[0].find(class_='value').get_text())
print(get_all_header[0].find(class_='address').get_text().replace('\ue92f',''))
print(get_all_header[0].find(class_='heading').get_text().split('$')[0])
# print(get_all_header[0].find_all(class_='label'))
print(get_all_header[0].find(class_='featured').get_text())

$130,000
Kandaek, Prasat Bakong, Siem Reap
Villa
Villa for Sale in Siem Reap-Kon Deak Borey Tourism siem reap


In [4]:
product_info = []

In [5]:
def get_product_info(url):
    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get(url)
    headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36'}
    response = requests.get(url, headers=headers)
    data = bs(response.content, 'html.parser')


    # query all headers
    get_all_header_no_nest = data.find_all(class_='item')

    time.sleep(20)
    for item in get_all_header_no_nest:
        name = item.find(class_='featured') if item.find(class_='featured') == None else item.find(class_='featured').get_text()
        loaction = item.find(class_='address') if item.find(class_='address') == None else  item.get_text().replace('\ue92f','')
        category = item.find(class_='heading') if item.find(class_='heading') == None else item.find(class_='heading').get_text().split('$')[0]        
        price = item.find(class_='value') if item.find(class_='value') == None else item.find(class_='value').get_text()
        infos = str(item.find_all(class_='label'))
        product_info.append({'name': name, 'location' : loaction, 'category' : category, 'price' : price, 'infos' : infos})

In [6]:
url_for_loop = 'https://www.realestate.com.kh/buy/?page='

for page_number in range(1, 51, 1):
    get_product_info('https://www.realestate.com.kh/buy/?page='+str(page_number))


# get_product_info('https://www.realestate.com.kh/buy/?page=1')

[WDM] - Downloading: 100%|██████████████| 7.26M/7.26M [00:06<00:00, 1.24MB/s]
/tmp/ipykernel_39830/642083662.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
product_info


[{'name': None,
  'location': None,
  'category': None,
  'price': None,
  'infos': '[]'},
 {'name': 'Villa for Sale in Siem Reap-Kon Deak Borey Tourism siem reap',
  'location': 'Villa for Sale in Siem Reap-Kon Deak Borey Tourism siem reapFeaturedFor sale\n\nVilla$130,000  Kandaek, Prasat Bakong, Siem Reap\ue920 Bedrooms: 33\ue91f Bathrooms: 33\ue93f Land area: 240m²240m² Updated: 1\xa0week agoDaBest Properties Cambodia Co., Ltd',
  'category': 'Villa',
  'price': '$130,000',
  'infos': '[<span class="label">Bedrooms: 3</span>, <span class="label">Bathrooms: 3</span>, <span class="label">Land area: 240m²</span>]'},
 {'name': 'Land for Sale in\u200b Krong Siem Reap-Sangkat Chreav ',
  'location': 'Land for Sale in\u200b Krong Siem Reap-Sangkat Chreav FeaturedFor saleLand$59,150  Chreav, Siem Reap, Siem Reap\ue93f Land area: 455m²455m² Updated: 1\xa0week agoDaBest Properties Cambodia Co., Ltd',
  'category': 'Land',
  'price': '$59,150',
  'infos': '[<span class="label">Land area: 455m²

# Saving and Loading data

In [8]:
def save_data(title,data):
    with open(title, "w",encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=3)

In [9]:
def load_data(title):
    with open(title,encoding='utf-8') as f:
        return json.load(f)

In [12]:
realestate_kh = "./data/realestate_kh.json"
save_data(realestate_kh,product_info)
realestate_kh_list = load_data(realestate_kh)

In [13]:
realestate_kh_list

[{'name': None,
  'location': None,
  'category': None,
  'price': None,
  'infos': '[]'},
 {'name': 'Villa for Sale in Siem Reap-Kon Deak Borey Tourism siem reap',
  'location': 'Villa for Sale in Siem Reap-Kon Deak Borey Tourism siem reapFeaturedFor sale\n\nVilla$130,000  Kandaek, Prasat Bakong, Siem Reap\ue920 Bedrooms: 33\ue91f Bathrooms: 33\ue93f Land area: 240m²240m² Updated: 1\xa0week agoDaBest Properties Cambodia Co., Ltd',
  'category': 'Villa',
  'price': '$130,000',
  'infos': '[<span class="label">Bedrooms: 3</span>, <span class="label">Bathrooms: 3</span>, <span class="label">Land area: 240m²</span>]'},
 {'name': 'Land for Sale in\u200b Krong Siem Reap-Sangkat Chreav ',
  'location': 'Land for Sale in\u200b Krong Siem Reap-Sangkat Chreav FeaturedFor saleLand$59,150  Chreav, Siem Reap, Siem Reap\ue93f Land area: 455m²455m² Updated: 1\xa0week agoDaBest Properties Cambodia Co., Ltd',
  'category': 'Land',
  'price': '$59,150',
  'infos': '[<span class="label">Land area: 455m²